# ETL Pipeline (PySpark) — Local
Este notebook orquestra leitura (ingest), normalização e transformações simples.
Edite os caminhos e funções conforme seu dataset.

In [1]:
# De notebooks/ sobe 1 nível até a raiz do projeto
import sys
from pathlib import Path
ROOT = Path().resolve().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.etl.utils import get_spark, RAW, INTERIM, PROCESSED
spark = get_spark("ETLLocalNotebook")
spark.version, RAW, PROCESSED


('3.5.3',
 WindowsPath('C:/Users/fred/meu_projeto_etl/data/raw'),
 WindowsPath('C:/Users/fred/meu_projeto_etl/data/processed'))

In [2]:
# Ingest: ler todos os CSVs de data/raw
from src.etl.ingest import read_csv_folder, basic_normalize
df_raw = read_csv_folder(spark, RAW)
df_raw.printSchema()
df_raw.show(5, truncate=False)

# Normalizações básicas
df_norm = basic_normalize(df_raw)
df_norm.printSchema()
df_norm.show(5, truncate=False)

root
 |-- Variable: string (nullable = true)
 |-- Description: string (nullable = true)

+-----------------------+--------------------------------------------------+
|Variable               |Description                                       |
+-----------------------+--------------------------------------------------+
|actualdpd_943P         |Days Past Due (DPD) of previous contract (actual).|
|actualdpdtolerance_344P|DPD of client with tolerance.                     |
|addres_district_368M   |District of the person's address.                 |
|addres_role_871L       |Role of person's address.                         |
|addres_zip_823M        |Zip code of the address.                          |
+-----------------------+--------------------------------------------------+
only showing top 5 rows

root
 |-- variable: string (nullable = true)
 |-- description: string (nullable = true)

+-----------------------+--------------------------------------------------+
|variable               |de

In [ ]:
# Transforms: exemplos
from src.etl.transform import drop_empty_rows, cast_columns, simple_enrich

# Exemplo: defina colunas críticas e casts conforme seu caso
critical = []  # ex.: ['id', 'data']
casts = {}     # ex.: {'id': 'int', 'valor': 'double'}

df_t = drop_empty_rows(df_norm, subset=critical)
df_t = cast_columns(df_t, casts)
df_t = simple_enrich(df_t)
df_t.show(5, truncate=False)
df_t.printSchema()

In [ ]:
# Persistência: salva resultado processado em CSV (com header)
# Ajuste o caminho/particionamento conforme necessidade.
(
    df_t.coalesce(1)
    .write.mode('overwrite')
    .option('header', True)
    .csv(str(PROCESSED / 'dataset_processado'))
)
print('OK — dados salvos em', PROCESSED / 'dataset_processado')

In [ ]:
# Encerramento opcional da sessão
spark.stop()
print('SparkSession finalizada')